# Haqathon: Traveling Salesman Problem using Variational Algorithms

Wenyang Qian    
March 21st: 9:00 - 20:00

The **Traveling Salesman Problem**, or **TSP** for short, is one of the most intensively studied problems in computational mathematics. 

People have devoted to the history, applications, and current research of this challenge of finding the shortest route visiting each member of a collection of locations and returning to your starting point.

For over a century, TSP has inspired hundreds of works and dozens of algorithms, of both exact and heuristic approaches. Today, the TSP has become so quintessential in modern computing that it is commonly considered the prototypical NP-Hard combinatorial optimization problem, possessing far-reaching impact on countless applications in science, industry and society. See website [here](https://www.math.uwaterloo.ca/tsp/) from University of Waterloo and others for more information about TSP.

Today, we aim to come up with solutions to solve the TSP using quantum variational algorithms. 



<img src="image/tsp.webp" width=400>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import transpile
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.visualization import plot_histogram
from qiskit.primitives import Estimator, Sampler, StatevectorEstimator, StatevectorSampler
import qiskit_aer
import qiskit_algorithms
import qiskit

print('Qiskit version:', qiskit.version.get_version_info())

## 1. Problem Formulation of TSP

The TSP data that people typically use is from [TSPLIB](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/index.html), stored in ".tsp" text files. 

For simplicity, we focus on **symmetric TSP** where going from A to B is the same as going from B to A. I have downloaded these files and added some other files under "data" directory.

One can use "TSPParser" class (credit: [tsartsaris](https://github.com/tsartsaris/TSPLIB-python-parser)) to convert the text file to Python data and also visualize it! You may look at the text files and play with the one you like. Here is a map in Berlin.

In [ ]:
from HaqatonTSP.parser import TSPParser
TSPParser(filename="data/berlin52.tsp", plot_tsp=True)

As you can see, TSP problem is essentially best routing problem with graphs! The cities are the nodes and the roads between them become the edges of the graph. The traveling distances of each road is the corresponding edge weight. To understand TSP, we need to under Graph theory.

In graph theory, it is convenient to introduce the concept of **adjacency matrix** to describe the connectivity inside the graph. An adjacency matrix $M$ is a square matrix used to represent a finite graph. The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph. 

Specifically, we also store edge weights directly int he elements of an adjacency matrix. Here, we use $M_{ij} =0$ to indicate there is no edge connecting city $i$ and $j$, and $M_{ij} = d$ to indicate there is a road of distance $d$ connecting city $i$ and $j$.

Here are two examples:

<img src="image/tsp_case1.png" width=350>
The adjacency matrix for the above graph should be: 
$ \quad\quad M = \begin{pmatrix}
    0 & 2 & 1 & 0 \\
    2 & 0 & 0 & 3 \\
    1 & 0 & 0 & 2 \\
    0 & 3 & 2 & 0 
    \end{pmatrix} 
$

<img src="image/tsp_case2.png" width=350>
The adjacency matrix for the above graph should be: 
$ \quad\quad M = \begin{pmatrix}
    0 & 2 & 1 & 3 \\
    2 & 0 & 3 & 3 \\
    1 & 3 & 0 & 2 \\
    3 & 3 & 2 & 0 
    \end{pmatrix} 
$

Note, we start our city index from 0, for Python is 0-indexed. 

To make it easier for you, I wrote for you the following **TSPGraph** class to build and visualize the TSP from any adjacency matrix. The above examples can be built easily. When rendering the city indices might switch places for optimal presentation.

In [ ]:
from HaqatonTSP.tsp import TSPGraph

# use tha adjaceny matrix above
M = \
[[0,2,1,0], 
 [2,0,0,3], 
 [1,0,0,2], 
 [0,3,2,0]]

tsp1 = TSPGraph(num_nodes=4, adj_matrix=M)
tsp1.draw()

In [ ]:
tsp1.get_sols(keep_unique=False)

In [ ]:
# use tha adjaceny matrix above

M = \
[[0,2,1,3], 
 [2,0,3,3], 
 [1,0,3,2], 
 [3,3,2,0]]

tsp2 = TSPGraph(num_nodes=4, adj_matrix=M)
tsp2.draw()

One can also use the class to get a random TSP graph, by **not specifying** the adjacency matrix. Additioanlly, one can specficy the connectivity of the edges (edge_freq) with 1 being fully connected and 0 being fully disjointed, and max_weight for the maximum edge weight allocated. One can also play with seed values to generate a variety of graphs.

In [ ]:
tsp3 = TSPGraph(num_nodes=6, seed=0, edge_freq=1.0, max_weight=100)
tsp3.draw()

You can use **get_adj_matrix()** function to print out the adjacency matrix.

In [ ]:
tsp3.get_adj_matrix()

In [ ]:
tsp4 = TSPGraph(num_nodes=6, seed=42, edge_freq=0.7, max_weight=500)
tsp4.draw()

In [ ]:
tsp4.get_adj_matrix()

## 2. Exact Classical Solution

Here, you can solve the TSP problem for both the optimal distance and route using bruteforce algorithm provided by the class.

Note the algorithm scales worse than exponential, since $\mathcal{O(n!)} > \mathcal{O}(2^n)$, for a $n$-node TSP graph. So, it will fail terribly (taking forever to run) when $n$ is large.

In [ ]:
tsp1.get_sols()

The **get_sols()** function returns the best distance and a list of possible route path. For example, the first TSP graph has a best distance of 8, with possible routes (0,2,3,1) representing 0 -> 2 -> 3 -> 1 -> 0 and (0,1,3,2) representing 0 -> 1 -> 3 -> 2 -> 0. Importantly, for TSP, there could be multiple best route path correspond to the same one best distance. Note, it is assumed that you will return to the starting city.

One can also use the **draw_with_bf_sol()** to draw the best path on graph.

In [ ]:
tsp1.draw_with_bf_sol()

Now, you can experiment with the other graphs.

In [ ]:
tsp2.get_sols()

In [ ]:
tsp2.draw_with_bf_sol()

In [ ]:
tsp3.get_sols()

In [ ]:
tsp3.draw_with_bf_sol()

In [ ]:
tsp4.get_sols()

In [ ]:
tsp4.draw_with_bf_sol()

Note it is possible to have a TSP **without a best distance**, if an edge between two nodes are not availble. For example, you can set freqence of the edge to be very low. Because in our setup, each edge is only allowed to be used once.

In [ ]:
tsp5 = TSPGraph(num_nodes=6, seed=0, edge_freq=0.5, max_weight=100)
tsp5.draw()

In [ ]:
tsp5.get_sols()

## 3. Approximate Quantum Solution using Variational Algorithms

Here, you will solve the TSP problem using quantum variational algorithms, such as QAOA or VQE.

Some useful resources:

* Qiskit tutorial, Max-cut and TSP, [GitHub](https://github.com/qiskit-community/qiskit-optimization/blob/main/docs/tutorials/06_examples_max_cut_and_tsp.ipynb)

* Ising formulations of many NP problems, [1302.5843](https://arxiv.org/abs/1302.5843)   

* Comparative Study of Variations in Quantum Approximate Optimization Algorithms for the Traveling Salesman Problem, [2307.07243](https://arxiv.org/abs/2307.07243)

...

You can see that like Max-cut problem you learned in class, TSP can also be formulated using Ising Model, which is basically some weighted sum of Pauli-$Z$ and Pauli-$ZZ$ operators. 

### a. TSP Problem Formulation as Optimization Probelm
First step is forumlate TSP as an **optimization problem**. Optimization problem allows us to go much beyond exact solution by bruteforce. The optimization problem comes with **three important parts**.

1. What are the quantum state? How are solutions represented?
   
2. What is the cost operator for representing the TSP graph that we try to minimize?

3. What are the constraints for the TSP problem? Why are the contraints necessary?

#### a.1. Quantum state to represent solution

In this graph formulation of the TSP, any valid cycle, be it minimum or not, can be represented by a visiting order or a permutation of integers, such as $\{0, 1, ..., n-1\}$, where the integers are the city indices starting at 0 for a total of $n$ cities. 

Alternatively, the visiting order on a TSP graph can be conveniently described by a sequence of **binary decision variables**, $x_{i,t}$, indicating whether the city-$i$ is visited at time $t$. If $x_{i,t}=1$ then the city-$i$ is visited at $t$, otherwise the city is not visited by the traveling salesman. 

Naively, to fully describe the solution to a $n$-city TSP, a total of $n^2$ binary variables is needed in this representation. 

Alternatively, this ``one-hot" representation of binary decision variables can be written collectively in either **matrix** or **flattened array format** for numerical implementation. For instance, a valid Hamiltonian cycle of permutation $x=(0, 1, 2, 3)$, is translated into binary decision variables $x$ as
$$
\begin{equation}
\begin{aligned}
    x =(0, 1, 2, 3) 
    \equiv
    \begin{pmatrix}
    1 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 
    \end{pmatrix} 
    \equiv 1000010000100001 ,
\end{aligned}
\end{equation}
$$
where the matrix row index represents each city index, and the column index represents each time instance. 

This is then very suitable for our quantum state. Take some time to think this through. If needed, you can also look at the references.

#### a.2. TSP cost operator

With binary decision variables $x$, a true solution to an $n$-city TSP can be found by finding an $x$ that minimizes the following cost function,
$$
\begin{align}
    C_\mathrm{dist}(x) = \sum_{0\leq i,j<n} \omega_{ij} \sum_{t=0}^{n-1} x_{i,t} x_{j,t+1},
\end{align}
$$
where $\omega_{ij}$ is the distance (or edge weight in the undirected graph) between city-$i$ and city-$j$.

Note: in symmetric TSP, $\omega_{ij} = \omega_{ji}$ and $\omega_{ii}=0$.

Now it is your time to implement this using what you learned from the Max-cut problem. 

You function should take in an adjaceny matrix (a $2^n$ by $2^n$ matrix) for a TSP graph and return the cost operator for this TSP in terms of Pauli strings. Importantly as a sanity check, you should end up with only Pauli-Z and Pauli-ZZ operators.

In [ ]:
def build_TSP_cost_operator(num_nodes, TSP_adj_matrix):
    '''
    Return the TSP cost operator that we try to minimize.
    '''


    
    
    return operator

#### a.3. TSP constraint equations

You should quickly see that not all decision variables are valid. Some of those are not a possible route and some of those violate causality. 

For example, each row of the matrix representation of the decision variables must have 1 appearing once. Otherwise, the TSP person is at the same city all the time. 

Likewise, each column of the matrix representation of the decision variables must have 1 appearing once. Otherwise, the TSP person is simultaneous at all the cities!!

What do you think is the constraints here?

Yes, the matrix must faithly represent a permutation matrix. Much in the same spirit as **Soduku**, the decision variable must has exactly 1 in any row and any column!

For this, we categorize the **decision variable $x$ into three categories**:

$$
\begin{align}
    x &= \begin{cases}
        \textbf{true}, & \text{$x$ is a permutation and gives the shortest path,}\\
        \textbf{false}, & \text{$x$ is a permutation but does not give the shortest path,}\\
        \textbf{invalid}, & \text{$x$ is not a permutation,}
    \end{cases}
\end{align}
$$

Write a few $x$ and determine if it is valid (true + false) or invalid.

Then, how do we materialize this fact in our setup? 

Since the cost function itself does not forbid invalid solutions in general, additional constraint conditions must be satisfied for a valid Hamiltonian cycle, such as
$$
\begin{align}
    \sum_{i=0}^{n-1} x_{i,t} &= 1\quad \text{for $t=0,1,\cdots,n-1$}  \\
    \sum_{t=0}^{n-1} x_{i,t} &= 1\quad \text{for $i=0,1,\cdots,n-1$} 
\end{align}
$$
where the former equation forbids multiple cities visited by the traveler at the same time, and latter equation forbids revisiting the same city. 

To formulate the TSP as a minimum-optimization problem, these constraint conditions are conveniently incorporated as the penalty terms, such that the combined cost function, $C(x)$ becomes,
$$
\begin{align}
    C(x) =&\, C_\mathrm{dist}(x) + \lambda C_\mathrm{penalty}(x) \\ =&\sum_{0\leq i,j<n} \omega_{ij} \sum_{t=0}^{n-1} x_{i,t} x_{j,t+1}
    + \lambda \bigg\{\sum_{t=0}^{n-1}\Big(1-\sum_{i=0}^{n-1} x_{i,t}\Big)^2 
    + \sum_{i=0}^{n-1}\Big(1-\sum_{t=0}^{n-1} x_{i,t}\Big)^2\bigg\},
\end{align}
$$
where $\lambda$ is the weight factor of the penalty term, serving as the Lagrange multiplier. $\lambda$ should be positive and sufficiently large. 

It is easy to see bit string $x$ gives the minimum of $C(x)$ **if and only if** $x$ is a true solution to the given TSP. 

Now our problem is equivalent to finding an $x^*$ that **minimizes** $C(x)$, i.e. $x^* = {\arg \min}\, C(x)$.

In [ ]:
def build_TSP_penalty_operator(num_nodes, lambda_factor):
    '''
    Return the TSP penalty operator that we try to enforce the constraint upto some weight lambda_factor.
    '''


    
    
    return operator

In [ ]:
def build_full_TSP_operator(num_nodes, TSP_adj_matrix, lambda_factor):
    '''
    Return the full TSP operator that we try to minimize.
    '''


    
    
    return operator

Make sure the operator you got is still an Ising operator.

Then you can use exact diagonalization to verify the best distance using numpy.



In [ ]:
# as an example using numpy to find eigenvalues

op     = build_full_TSP_operator(num_nodes, TSP_adj_matrix, lambda_factor)
op_mat = operator.to_matrix()

# print out the first 10 eigenvalues
sorted(np.linalg.eigvals(op_mat))[:10]

### b. Variational Ansatz for TSP

Congratulations!

Getting here is significant!!! Now, the floor is yours. Try VQE or QAOA setup that you learned to see if you can extract the optimal distance and route. **Try start with VQE**, as it is a simple heuristic approach.



In [ ]:

def run_VQE_on_TSP(tsp_operator, ansatz, optimizer, maxiter, shots):
    '''
    Run the quantum simulation and return the optimizer result and a list of expectation values for your ansatz

    Suggestion: try go to Max-cut notebook from previous class, see "maxcut_vqe_from_scratch" function, if this is not very clear.
    '''


    
    
    return opt_result, exp_val_list


# you can use the get_expectation function from our class.
def get_expectation(ansatz, params, observable, shots):
    assert ansatz.num_qubits == observable.num_qubits, f"ansatz qubits = {ansatz.num_qubits}, observable qubits = {observable.num_qubits}"
    assert len(params) == ansatz.num_parameters
    ##### ==================================
    # Write your solution in here.  
    circuit = ansatz.assign_parameters(params)
    expectation = estimate_with_shots(circuit, observable, shots)
    ##### ==================================
    return expectation
    
# you can use the estimator_with_shots function from our class.
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

def estimate_with_shots(circuit, observable, shots):
    ''' Using Estimator class from earlier Qiskit versions (will be deprecated soon)
        Still useful because one can implement shot simulation
        Ref: https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.Estimator
    '''
    _circuit = circuit.copy()
    _circuit.remove_final_measurements()
    shot_estimator = Estimator(options={"shots": shots})
    expectation = shot_estimator.run(_circuit, observable).result().values[0]
    return expectation

def estimate_with_shots_v2(circuit, observable):
    ''' Using StatevectorEstimator class from Qiskit 1.3 
        Ref: https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.StatevectorEstimator
    '''
    _circuit = circuit.copy()
    _circuit.remove_final_measurements()
    shot_estimator = StatevectorEstimator(shots=shots)
    pub = (circuit, [[observable]])
    expectation = shot_estimator.run([pub]).result()[0].data.evs[0]
    return expectation

To test it, you can use $n=3$ nodes TSP graph to begin with.

In [ ]:
num_nodes = 3

M = \
[[0,2,1], 
 [2,0,3], 
 [1,3,0]]

tsp = TSPGraph(num_nodes=num_nodes, adj_matrix=M)
tsp.draw()

In [ ]:
tsp.get_sols()

You know the cost has to be 6 and one possible route is (1,0,2). See if that is what you get. 

To obtain the route, remember to use sampling techqniue on the optimized ansatz to find state with maximal probability.

In [ ]:
def extract_TSP_route(ansatz, opt_res, shots):
    '''
    Sample the ansatz with optimization result and return best route and its probability.
    '''
    counts = sample_probability_dist(ansatz, opt_res, shots)




    
            
    return best_route, best_prob

# you can use sample_probability_dist function from our class.
def sample_probability_dist(ansatz, optimization_result, shots):
    num_qubits = ansatz.num_qubits
    ##### ==================================
    # Write your solution in here. 
    optimal_param = optimization_result.x
    qc = ansatz.assign_parameters(optimal_param)
    qc.measure_all()
    dist = sample_with_shots(qc, shots, num_qubits)
    ##### ==================================
    return dist
    
# you can use sample_with_shots function from our class.
def sample_with_shots(circuit, shots, num_qubits, export_prob=True):
    ''' Using StatevectorSampler class from Qiskit 1.3 
        Ref: https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.StatevectorSampler
    '''
    shot_sampler = StatevectorSampler(default_shots=shots)
    pub = (circuit)
    job = shot_sampler.run([pub], shots=shots)
    counts = job.result()[0].data.meas.get_counts()
    probs = {k: v/shots for k, v in counts.items()}
    return probs if export_prob else counts
    


In [ ]:
# for visualization

from qiskit.visualization import plot_histogram

counts = sample_probability_dist(ansatz, opt_res, shots)

plot_histogram(counts, number_to_keep=10)

In [ ]:
# extract

best_route, best_prob = extract_TSP_route(ansatz, opt_res, shots)
best_route, best_prob

Compare your solution with bruteforce ones

In [ ]:
print(tsp.get_sols())

tsp.draw_with_bf_sol()

### c. Optimize your setup to obtain Best Distance and Route

Now we will think of ways to improve our simulation, on more and more cities (successfully going up to 4-5 is already very good).

You can try with more shots, different optimizers, different initializations. If you used VQE, you should also try with QAOA. There are many things to improve this. Probably also good idea to read the paper.

At the end of the day, you are expected to come up with a **TSP solver**, that takes into any good TSP graph (adjacency matrix) and extract the best distance from quantum simulatoin! 

Have fun!!!

In [ ]:
def TSP_solver(num_nodes, tsp_adj_matrix, lambda_factor):
    '''
    Run quantum simulation to obtain the best distance of any tsp graph (tsp_adj_matrix)
    Return: best_dist, best_route, best_prob
    '''


    
    
    return best_dist, best_route, best_prob

In [ ]:
num_nodes = 3
max_weight = 100
edge_freq = 1.0

tsp = TSPGraph(num_nodes=num_nodes, edge_freq=edge_freq, max_weight=max_weight)
print(tsp.get_sols())

tsp_adj_matrix = tsp.get_adj_matrix()
print(tsp_adj_matrix)

tsp.draw_with_bf_sol()


In [ ]:
from time import time

n_trials = 5
for _ in range(n_trials):
    t0 = time()
    print(TSP_solver(num_nodes, tsp_adj_matrix, lambda_factor=max_weight), f", time cost = {time()-t0:.2f}s")

## 4. Evaluation matrix

Here, you will try to improve your quantum solver for TSP problem. The performance of your solver will be evaluated based on the following crtierions

| Crtierion    | Description |
| :-------- | :------- |
| Number of Cities (n)  | number of TSP cities    |
| Simulation Time (t) | time to perform quantum simulation     |
| Approximation Ratio (AR)    | simulated optimal distance divided by the exact optimal distance    |


The score $\mathrm{S}$ for each $n$ is
$$\mathrm{S}_n = \frac{100}{\mathrm{AR}\log(t)},$$
and the total score of your Haqaton is
$$\mathrm{S} = \sum_{n} n^2 \mathrm{S}_n. $$

Since it is an approximation optimization, you have several trials for your simulation. The team with the highest score $\mathrm{S}$ will be winner for this project.


In [ ]:
def score_n(sim_best_dist, bf_dist, sim_time, n):
    AR = (sim_best_dist/bf_dist)
    return n*n*100/AR/np.log2(sim_time)

In [ ]:
# enter your time here.
n = 3
sim_best_dist = 245.8641592590993
bf_dist = 217
sim_time = 6.29
print("Score =", score_n(sim_best_dist, bf_dist, sim_time, n))
